In [1]:
#!pip install cassandra-driver
from cassandra.cluster import Cluster
import pandas as pd
import numpy as np
from cassandra.cqlengine import columns
from cassandra.cqlengine.models import Model
from cassandra.cqlengine import models
from cassandra.cqlengine import connection
#esta libreria sincroniza la base de datos y crea la base de datos 	
from cassandra.cqlengine.management import sync_table

In [2]:
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

In [3]:
#leemos nuestro fichero csv 
#especificamos la ruta del fichero (para saber nuestra ruta podemos ejecutar el comando !ls similar a linux) 	
suicides = pd.read_csv('./suicidios.csv', sep = ',', thousands=b',', header=0)

In [4]:
#tenemos nuestro dataframe 	
suicides.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,2156624900,796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,2156624900,796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,2156624900,796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,2156624900,796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,2156624900,796,Boomers


In [5]:
#podemos ver que hay unas columnas algo extrañas	
suicides.columns

Index(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides/100k pop', 'country-year', 'HDI for year',
       ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation'],
      dtype='object')

In [6]:
suicides.rename({'suicides/100k pop':'suicides_by_100k_pob', 
                 'HDI for year':'HDI_for_year',
                 ' gdp_for_year ($) ': 'pib_for_year',
                 'gdp_per_capita ($)': 'pib_per_capita'}, axis=1, inplace=True)

In [7]:
#el implace es un reemplazo definitivo mas no una copia 	
suicides.head()

,country,year,sex,age,suicides_no,population,suicides_by_100k_pob,country-year,HDI_for_year,pib_for_year,pib_per_capita,generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,2156624900,796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,2156624900,796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,2156624900,796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,2156624900,796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,2156624900,796,Boomers


In [8]:

suicides.columns


Index(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides_by_100k_pob', 'country-year', 'HDI_for_year', 'pib_for_year',
       'pib_per_capita', 'generation'],
      dtype='object')

In [9]:
#¿que tipos de datos son nuestras columnas?
suicides.dtypes


country                  object
year                      int64
sex                      object
age                      object
suicides_no               int64
population                int64
suicides_by_100k_pob    float64
country-year             object
HDI_for_year            float64
pib_for_year              int64
pib_per_capita            int64
generation               object
dtype: object

In [ ]:
#tenemos una descripcion general de nuestra tabla con caracteristicas como el conteo, la media, la desviacion estandar, el maximo y el minimo etc...
suicides.describe()

In [10]:
#vamos a definir nuestros valores vacios (NaN), el .sum() hace una sumatoria de estos valores vacios 	
print(f"Existen {np.isnan(suicides.year).sum()} valores nan para la variable 'year'")
print(f"Existen {np.isnan(suicides.suicides_no).sum()} valores nan para la variable 'suicides_no'")
print(f"Existen {np.isnan(suicides.population).sum()} valores nan para la variable 'population'")
print(f"Existen {np.isnan(suicides.suicides_by_100k_pob).sum()} valores nan para la variable 'suicides_by_100k_pob'")
print(f"Existen {np.isnan(suicides.HDI_for_year).sum()} valores nan para la variable 'HDI_for_year'")
print(f"Existen {np.isnan(suicides.pib_per_capita).sum()} valores nan para la variable 'pib_per_capita'")

Existen 0 valores nan para la variable 'year'
Existen 0 valores nan para la variable 'suicides_no'
Existen 0 valores nan para la variable 'population'
Existen 0 valores nan para la variable 'suicides_by_100k_pob'
Existen 19456 valores nan para la variable 'HDI_for_year'
Existen 0 valores nan para la variable 'pib_per_capita'


In [11]:
#eliminamos esta variable axis 1 para nivel de columna y implace para eliminar directamente y no copiar o replicar 
suicides.drop('HDI_for_year', axis=1, inplace=True)

In [12]:
suicides.columns

Index(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides_by_100k_pob', 'country-year', 'pib_for_year',
       'pib_per_capita', 'generation'],
      dtype='object')

In [13]:
# ## Creamos nuestro modelo de datos, comenzamos creando el keyspace y la tabla

# In[18]:


session.execute(
    """CREATE KEYSPACE IF NOT EXISTS PROYECTO WITH replication = {'class': 'SimpleStrategy', 
                                                    'replication_factor' : 1}; """
)

In [14]:
#usamos nuestro keyspace es decir que tenemos nuestra variable session apuntando a ese keyspace	
session.set_keyspace("proyecto")

In [15]:
class Suicide(Model):
    country = columns.Text(primary_key = True)
    year = columns.Integer(primary_key = True)
    sex  = columns.Text(primary_key = True)
    age = columns.Text(primary_key = True, clustering_order="ASC")
    suicides_no = columns.Integer()
    population = columns.BigInt()
    suicides_by_100k_pob = columns.Float()
    country_year = columns.Text()
    pib_for_year = columns.BigInt()
    pib_per_capita = columns.BigInt()
    generation = columns.Text()

In [16]:
#debemos registrar nuestra conexion le damos un nombre y le asignamos nuestra variable sesion 
connection.register_connection('cluster3', session=session)
sync_table(Suicide, keyspaces=['proyecto'],connections=['cluster3'])

/home/cristian/anaconda3/lib/python3.12/site-packages/cassandra/cqlengine/management.py:544: UserWarning: CQLENG_ALLOW_SCHEMA_MANAGEMENT environment variable is not set. Future versions of this package will require this variable to enable management functions.
  warnings.warn(msg)


In [17]:
#verificamos que nuestra tabla fue generada exitosamente 	
for row in session.execute("""select column_name, kind, type from system_schema.columns WHERE keyspace_name = 'proyecto' ;"""):
    print(row)

{'column_name': 'age', 'kind': 'clustering', 'type': 'text'}
{'column_name': 'country', 'kind': 'partition_key', 'type': 'text'}
{'column_name': 'country_year', 'kind': 'regular', 'type': 'text'}
{'column_name': 'generation', 'kind': 'regular', 'type': 'text'}
{'column_name': 'pib_for_year', 'kind': 'regular', 'type': 'bigint'}
{'column_name': 'pib_per_capita', 'kind': 'regular', 'type': 'bigint'}
{'column_name': 'population', 'kind': 'regular', 'type': 'bigint'}
{'column_name': 'sex', 'kind': 'clustering', 'type': 'text'}
{'column_name': 'suicides_by_100k_pob', 'kind': 'regular', 'type': 'float'}
{'column_name': 'suicides_no', 'kind': 'regular', 'type': 'int'}
{'column_name': 'year', 'kind': 'clustering', 'type': 'int'}


In [ ]:
# ### Vamos a insertar los datos de suicidio en la tabla

In [18]:
#vamos a 'volcar' los datos de nuestro dataframe en la tabla cassandra que acabamos de crear, esto lo haremos de manera secuencial fila a fila 
#por medio de un bucle for
#la variable index viene por defecto en el dataframe pero esta no se va a agregar en la base de datos 
#la '%s' equivale a los values fuera de la sentencia cql
#podemos limitar la cantidad de tuplas o registros que queremos pasar a nuestra base de datos con el metodo del dataframe suicides.head(n)
for index,value in suicides.iterrows():
    session.execute(""" INSERT INTO proyecto.suicide (country, year, sex, age, country_year, generation,  
                            pib_for_year, pib_per_capita, population,  suicides_by_100k_pob, 
                            suicides_no)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""",
                   (value['country'], value['year'], value['sex'], value['age'], 
                   value['country-year'], value['generation'], value['pib_for_year'], value['pib_per_capita'], 
                   value['population'], value['suicides_by_100k_pob'], value['suicides_no']) )

In [19]:
#podemos saber que tipo de variable es nuestra variable rows con el comando type(my_rows)	
all_rows = session.execute("""SELECT * FROM proyecto.suicide""")

for row in all_rows[:20]:
    print(row)

{'country': 'Ecuador', 'year': 1985, 'sex': 'female', 'age': '15-24 years', 'country_year': 'Ecuador1985', 'generation': 'Generation X', 'pib_for_year': 17149094590, 'pib_per_capita': 2238, 'population': 897700, 'suicides_by_100k_pob': 6.460000038146973, 'suicides_no': 58}
{'country': 'Ecuador', 'year': 1985, 'sex': 'female', 'age': '25-34 years', 'country_year': 'Ecuador1985', 'generation': 'Boomers', 'pib_for_year': 17149094590, 'pib_per_capita': 2238, 'population': 600800, 'suicides_by_100k_pob': 5.159999847412109, 'suicides_no': 31}
{'country': 'Ecuador', 'year': 1985, 'sex': 'female', 'age': '35-54 years', 'country_year': 'Ecuador1985', 'generation': 'Silent', 'pib_for_year': 17149094590, 'pib_per_capita': 2238, 'population': 692800, 'suicides_by_100k_pob': 2.1700000762939453, 'suicides_no': 15}
{'country': 'Ecuador', 'year': 1985, 'sex': 'female', 'age': '5-14 years', 'country_year': 'Ecuador1985', 'generation': 'Generation X', 'pib_for_year': 17149094590, 'pib_per_capita': 2238,

In [20]:
for row in all_rows[0:20]:
    print(f"""En {row['country']} en el año {row['year']} hubo {row['suicides_no']} suicidios de genero {row['sex']} con edades comprendidas en {row['age']} """)

En Ecuador en el año 1985 hubo 58 suicidios de genero female con edades comprendidas en 15-24 years 
En Ecuador en el año 1985 hubo 31 suicidios de genero female con edades comprendidas en 25-34 years 
En Ecuador en el año 1985 hubo 15 suicidios de genero female con edades comprendidas en 35-54 years 
En Ecuador en el año 1985 hubo 4 suicidios de genero female con edades comprendidas en 5-14 years 
En Ecuador en el año 1985 hubo 4 suicidios de genero female con edades comprendidas en 55-74 years 
En Ecuador en el año 1985 hubo 2 suicidios de genero female con edades comprendidas en 75+ years 
En Ecuador en el año 1985 hubo 104 suicidios de genero male con edades comprendidas en 15-24 years 
En Ecuador en el año 1985 hubo 70 suicidios de genero male con edades comprendidas en 25-34 years 
En Ecuador en el año 1985 hubo 57 suicidios de genero male con edades comprendidas en 35-54 years 
En Ecuador en el año 1985 hubo 7 suicidios de genero male con edades comprendidas en 5-14 years 
En Ec

In [21]:
# Ejecutar la consulta
num_paises = session.execute("""SELECT country FROM suicide""")

# Acumular resultados en una lista
rows = [row for row in num_paises]

# Crear el DataFrame
df_num_paises = pd.DataFrame(rows, columns=["country"])


In [22]:
df_num_paises.head()

,country
0,Ecuador
1,Ecuador
2,Ecuador
3,Ecuador
4,Ecuador


In [23]:
suicide_greater_250 = session.execute(""" SELECT * FROM suicide WHERE suicides_no > 250""")

InvalidRequest: Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"

In [24]:
#de este modo no tendriamos problema 	
suicide_greater_250 = session.execute(""" SELECT * FROM suicide WHERE suicides_no > 250 ALLOW FILTERING""")
rows = [row for row in suicide_greater_250]

df_suicide_greater = pd.DataFrame(rows)

In [25]:
df_suicide_greater.head()

,country,year,sex,age,country_year,generation,pib_for_year,pib_per_capita,population,suicides_by_100k_pob,suicides_no
0,Spain,1985,female,55-74 years,Spain1985,G.I. Generation,180302412231,5009,3683300,7.520000,277
1,Spain,1985,male,35-54 years,Spain1985,Silent,180302412231,5009,4471200,11.120000,497
2,Spain,1985,male,55-74 years,Spain1985,G.I. Generation,180302412231,5009,3115900,20.030001,624
3,Spain,1985,male,75+ years,Spain1985,G.I. Generation,180302412231,5009,691500,44.110001,305
4,Spain,1986,female,55-74 years,Spain1986,G.I. Generation,250638463467,6919,3738800,8.050000,301


In [ ]:

#creamos un indice secundario 
session.execute(""" CREATE INDEX IF NOT EXISTS sex_suicide ON suicide (sex)""")

In [ ]:
#con nuestro indice secundario ya podemos filtrar por campos no pertenecientes a la primary key 	
female_suicides = session.execute(""" SELECT * FROM suicide WHERE sex = 'female'""")

In [ ]:

rows = [row for row in female_suicides]

df_female_suicides = pd.DataFrame(rows)

In [ ]:
df_female_suicides.head()

In [ ]:
# #### ¿Cómo crear una vista materializada para poder filtrar por el campo generation y conteniendo unicamente los registros de paises con poblacion mayor a 10 millones de personas?
# 

# In[43]:

#creamos nuestra vista materializada es decir nuestra tabla proveniente de una consulta 
#las vistas materializadas no pueden tener nulos en las primary keys 
#le damos permiso a nuestra variable generacion para que se pueda filtrar, el resto de variables ya son primary key 
session.execute("""
    CREATE MATERIALIZED VIEW suicides_by_generation AS
        SELECT * FROM proyecto.suicide
        WHERE generation is not null and country is not null and 
        year is not null and age is not null and sex is not null and
        population > 10000000
        PRIMARY KEY (generation, country, year, age, sex)
    WITH comment='Vista materializada de generation';
""")

In [ ]:
#consultamos nuestra vista materializada como si fuera una tabla mas 
generation_x = session.execute("""SELECT * FROM suicides_by_generation WHERE generation = 'Generation X'""")
